In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import s3fs
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import numpy as np
from netCDF4 import Dataset
from matplotlib.colors import LogNorm
import matplotlib.path as mpath
from functions import compute_ivt,to_nc,count_ARs
from read_pangeo import read_pangeo, read_to_detect
from IPython import display

In [4]:
start_year =2085
end_year =2100
experiment_id = 'ssp585'

if experiment_id == 'ssp245' or experiment_id == 'ssp585' or experiment_id == 'historical':
    dset = read_to_detect(start_year = start_year, end_year = end_year, experimentid = experiment_id)

elif experiment_id == 'ssp370':
    s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", client_kwargs=dict(endpoint_url="https://rgw.met.no"))
    s3path = list([
        'escience2022/Remy/va_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20810101-20901231.nc',
        'escience2022/Remy/va_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20910101-21001231.nc',
        'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20810101-20901231.nc',
        'escience2022/Remy/hus_day_NorESM2-LM_ssp370_r1i1p1f1_gn_20910101-21001231.nc',
    ])

    sopenlist=[s3.open(ss) for ss in s3path]
    dset = (xr.open_mfdataset(sopenlist)).sel(time = slice(str(start_year)+"-01-01", str(end_year)+"-12-31"))
    print('ssp370')



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [5]:
dset = dset.squeeze()
dset

<xarray.Dataset>
Dimensions:    (time: 5840, plev: 8, lat: 96, lon: 144, bnds: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(20, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(20, 2), meta=np.ndarray>
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * time       (time) object 2085-01-01 12:00:00 ... 2100-12-31 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(10, 2), meta=np.ndarray>
    member_id  <U8 'r1i1p1f1'
Dimensions without coordinates: bnds
Data variables:
    hus        (time, plev, lat, lon) float32 dask.array<chunksize=(10, 8, 20, 20), meta=np.ndarray>
    va         (time, plev, lat, lon) float32 dask.array<chunksize=(10, 8, 20, 20), meta=np.ndarray>
Attributes: (12/50)
    nominal_resolution:        250 km
    grid_label:                gn
    branch_method:             Hybrid-restart from year 2015-01-01 of historical
    parent_activity_id:        CMIP
    source_type:               AOGCM AER BGC
    model_id:                  NorESM2-LM
    ...                        ...
    source_id:                 NorESM2-LM
    version_id:                v20191108
    branch_time_in_child:      0.0
    contact:                   Please send any requests or bug reports to nor...
    mip_era:                   CMIP6
    intake_esm_dataset_key:    ScenarioMIP.NCC.NorESM2-LM.ssp585.day.gn

In [6]:
vas = dset.va
hus = dset.hus
plev = dset.plev
lat_ = dset.lat
lon_ = dset.lon

In [7]:
ivt = compute_ivt(hus,vas,plev)
dset.close()

<xarray.DataArray 'hus' (time: 5840, plev: 8, lat: 96, lon: 144)>
dask.array<getitem, shape=(5840, 8, 96, 144), dtype=float32, chunksize=(20, 8, 20, 20), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float64 -90.0 -88.11 -86.21 -84.32 ... 86.21 88.11 90.0
  * lon        (lon) float64 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
  * plev       (plev) float64 1e+05 8.5e+04 7e+04 5e+04 ... 1e+04 5e+03 1e+03
  * time       (time) object 2085-01-01 12:00:00 ... 2100-12-31 12:00:00
    member_id  <U8 'r1i1p1f1'
Attributes:
    cell_measures:   area: areacella
    cell_methods:    time: mean
    comment:         Specific humidity is the mass fraction of water vapor in...
    history:         2019-12-19T08:42:03Z altered by CMOR: Converted units fr...
    long_name:       Specific Humidity
    original_name:   Q
    original_units:  kg/kg
    standard_name:   specific_humidity
    units:           1
<xarray.DataArray 'va' (time: 5840, plev: 8, lat: 96, lon: 144)>
dask.array<ge

In [9]:
ivt_ns = ivt.copy()
ivt_ns = xr.where(ivt_ns.lat<0,-ivt_ns,ivt_ns,True) # minus for southern hemisphere (positive toward the pole)
ivt_ns_pos = xr.where(ivt_ns<0,ivt_ns*0,ivt_ns,True) # negative values = not poleward

In [10]:
# pre-computed 93th percentile IVT
q93 = xr.open_dataset('q93_2000.nc')
q93 = q93.rename({'__xarray_dataarray_variable__':'ivt'})

In [12]:
excess = ivt_ns_pos-q93

q93.close()
ivt_ns_pos.close()

ar_points = xr.where(excess>0,1,0)

In [13]:
out_ar = ar_points.copy()
ar_points.close()

#out_ar = out_ar.drop_vars(['quantile','member_id'])

out_loc = np.zeros((out_ar.ivt.shape[0],out_ar.ivt.shape[1],out_ar.ivt.shape[2])).astype(int)

res_lon = abs(lon_.values[1]-lon_.values[0])
res_lat = abs(np.min(np.diff(lat_.values)))

test_val = out_ar.ivt.values[:]

for tt in range(len(out_ar.time)):
    df_loc = test_val[:,tt,:]
    ll = plt.contour(df_loc,levels=[0,1])
    plt.close()
    for item in ll.collections:
        for i in item.get_paths():
            v = i.vertices
            crit = abs(np.max(v[:, 1])-np.min(v[:, 1]))
            if (crit>=20): # AR has to be at least 20 deg lat 
                xx=(v[:, 0]).astype(int)
                yy=(v[:, 1]).astype(int)
                for (x,y) in zip(xx,yy):
                    out_loc[y,tt,x] = 1
    

/tmp/ipykernel_1716/2524833928.py:15: UserWarning: No contour levels were found within the data range.
  ll = plt.contour(df_loc,levels=[0,1])


In [14]:
out_ar.ivt.values = out_loc.astype(bool)

In [15]:
out_ar.to_netcdf(str(experiment_id)+str(start_year)+str(end_year)+'_crit.nc')
out_ar.close()

In [16]:
AR = xr.open_dataset(str(experiment_id)+str(start_year)+str(end_year)+'_crit.nc')

In [17]:
ivt_ = AR.ivt
ivt = xr.concat([ivt_,ivt_[:,:,:5]],dim='lon')# add extra points for AR at the edge of the world

In [18]:
out_ar = ivt.copy()
out_ar.values[:] = out_ar.values[:]*0.0

def floodfill(indata):
    outloc = np.copy(indata)
    for x in range(indata.shape[0]):
        # for each latitude get limit longitudes of shapes
        ind_lim = np.where(indata[x,:]==1)
        for ii in ind_lim:
            if len(ii)>1: # if there is a shape at that longitude
                jj=0
                while jj<=len(ii)-2:
                    if abs(ii[jj]-ii[jj+1])>10: # avoid filling where only 1 point in shape
                        jj=jj+1
                    else:
                        outloc[x,ii[jj]:ii[jj+1]]=1
                        jj=jj+1
    return outloc


for k in range(ivt.shape[0]):
    matrix = ivt[k,:,:]
    tst = floodfill(matrix)
    out_ar[k,:,:] = tst
    
# add the values of the extra points to get ARs at the end of the world
out_ar[:,:,:5] = out_ar[:,:,:5]+out_ar[:,:,144:]
out_ar = out_ar[:,:,:144]
out_ar = xr.where(out_ar>1,1,out_ar)
out_ar = out_ar.astype(bool)

lat_ = AR.lat
lon_ = AR.lon

out_ar.to_netcdf(str(experiment_id)+str(start_year)+str(end_year)+'_AR_detection.nc')